In [1]:
!wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-1.tar.gz
!tar -xvf tasks_1-20_v1-1.tar.gz

--2019-09-26 06:40:26--  http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-1.tar.gz
Resolving www.thespermwhale.com (www.thespermwhale.com)... 69.65.3.213
Connecting to www.thespermwhale.com (www.thespermwhale.com)|69.65.3.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1282454 (1.2M) [application/x-gzip]
Saving to: ‘tasks_1-20_v1-1.tar.gz.3’

tasks_1-20_v1-1.tar 100%[===================>]   1.22M   582KB/s    in 2.2s    

2019-09-26 06:40:32 (582 KB/s) - ‘tasks_1-20_v1-1.tar.gz.3’ saved [1282454/1282454]

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/e

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/en/qa2_two-supporting-facts_train.txt
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/en/qa3_three-supporting-facts_test.txt
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/en/qa3_three-supporting-facts_train.txt
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tasksv11/en/qa4_two-arg-relations_test.txt
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header 

In [2]:
import numpy as np

In [3]:
f = open('tasksv11/en/qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()
f.close()
tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").replace("\t","").replace("1","").replace("5","").replace(".","")
                  .replace("10","").replace("0","").replace("2","").replace("8","").replace("?","").replace("3","")
                  .replace("4","").replace("7","").split(" ")[1:])

In [4]:
tokens[:7]

[['mary', 'moved', 'to', 'the', 'bathroom'],
 ['john', 'went', 'to', 'the', 'hallway'],
 ['where', 'is', 'mary', 'bathroom'],
 ['daniel', 'went', 'back', 'to', 'the', 'hallway'],
 ['sandra', 'moved', 'to', 'the', 'garden'],
 ['where', 'is', 'daniel', 'hallway'],
 ['john', 'moved', 'to', 'the', 'office']]

In [5]:
unique_vocab = set()
for sent in tokens:
    for word in sent:
        unique_vocab.add(word)

In [6]:
word2index = {}
for i,word in enumerate(unique_vocab):
    word2index[word]=i

In [7]:
def words2indices(sentence):
    """
    this func iterate through every word in sentence and append
    it's index to a list
            
            returns: list of index for each word in a sentence 
    Example:
    sentence = ['Omar','road','the','car'] converted to [23,1,12,9]
    """
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

In [8]:
len(unique_vocab)

19

In [9]:
vector_size = 10 #vector size for each word

In [10]:
one_hot = np.eye(len(unique_vocab))

# RNN
rnn cell has 3 weights matricies ,U called the embedding weight(the first weight to get multiplied by the one hot vector from the input 'words'),W is the weight matrix that get multiplied to the previous hidden state and V the weight matrix after the output of the RNN cell(will explain them later indetails). Let's begin with the easy part which is intializing the weights, it turned out that initializing weight affect the result of prediction so it is recommended to initialize the weights randomly in the interval from $\left[-\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}}\right]$ where $n$ is the number of incoming connections from the previous layer

In [11]:
U = (np.random.rand(len(unique_vocab),vector_size))
W = np.random.rand(vector_size)
V = (np.random.rand(vector_size,len(unique_vocab)))

In [12]:
W.shape

(10,)

We have almost reached half a way from the process of forward propagation, let dive in this process and see what actually happens

## Forward path

Input is one hot encoded  

### First
let begin with the process that happens with our first weight(embedding matrix), we simply extract the vector regards the index of the input word which in this case will be of size 10 as out vector is of size 10 and add it to the previous state. here's the equation for the following

$$s[t] = {U(x) + W.s(t - 1)}{}$$


 $$\begin{bmatrix} 18 & 55 & 1 & 2 & 1 & 4 & 11 & 44 & 33 & 100\\ 22 & 88 & 33 &71 & 3 & 0 & 1 & 7 & 3 & 33 \\ 20 & 19 & 9 & 0 & 2 & 4 & 10 & 13 & 44 & 31\end{bmatrix}$$

so for example if the word index is 1 then the vector related to it is the first row of the previous matrix.

### Second
a dot product of the s[t] with our last weight matrix V and this is simply our prediction layer i,e the vector [1,0,0,0,0] that translates to a certain word.



$$o[t] = {V.s(t)}{}$$


### softmax

In [13]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [14]:
def forward_prop(sentence):
    layers = list() # list for s[t] and o[t]
    layer = {} # dict indicating whether a layer is S[t] or o[t]
    loss = 0
    layer['s']  = np.zeros(vector_size) #s[0]
    layers.append(layer)
    for target_i in range(len(sentence)):
        # we computed the layer_1_2 here first it's odd right?
        # but if you though about it you will find that we now have a hidden
        # state s[0] and we need to use it to predict the next  word
        # we will use softmax here to keep values bet 0 and 1 since
        #remeber you want to predict sth like [1,0,0..]
        layer = {}
        layer['o'] = softmax(layers[-1]['s'].dot(V))
        loss += -np.log(layer['o'][sentence[target_i]])
        layer['s'] = U[sentence[target_i]] + layers[-1]['s'].dot(W)
        layers.append(layer)
    return layers, loss

Congraulations you have made it through forward progation!

### Backpropagation

In [17]:
learning_rate = 0.1

In [38]:
def back_prop():
    """
    This function will perform what is called back propagation through time
    we will walk through all layers and compute delta which is diff between
    predicted and actual label(magnitude and direction of change), weighted delta which 
    
    """
    for iter in range(30000):#still forwards prop
        sent = words2indices(tokens[iter%len(tokens)][1:])
        layers,loss = forward_prop(sent)
        for layer_i in reversed(range(len(layers))): # starting backprop
            layer = layers[layer_i]
            target = layers[layer_i - 1] #since if you see the output from the 'froward_prop' function, you will notice that
            # there's an extra state without an output layer in the end so that's why the target is to be the layer before the
            #last layer
            if(layer_i>0): # since layer[0] is just the s[0] so it's just the hidden
                #state at time zero not any predicted output
                layer['output_delta'] = layer['o'] - one_hot[target]
                new_weighted_delta = layer['output_delta'].dot(V.transpose())
                if(layer_i == len(layers)-1): # if last layer multiply by V
                    # since it's the weight matrix we need to stir the change 
                    # to achieve min loss
                    layer['hidden_delta'] = new_weighted_delta
                else:
                    #if not add weighted delta of the last layer with
                    # weighted delta of the layer exactly after the step
                    #you are in and multiply them the W matrix
                    layer['hidden_delta'] = new_weighted_delta + layers[layer_i+1]['hidden_delta'].dot(W.transpose())
            else: # if the first layer
                layer['hidden_delta'] = layers[layer_i+1]['hidden_delta'].dot(W.transpose())
        #----------------------------------------------------------------------------------------------------------
            # so we have been updating the matricies in the layers
            # but don't we need to update our actual weights accordingly
            # and by weights I mean the U, W and V as well as the s[0]
            # we assumed that it starts at 0,0,0,.. but is there a better
            # start in the direction of minimizing loss
            layers[0]['s'] -= layers[0]['s']*learning_rate / float(len(sent))
            for layer_i ,layer in enumerate(layers[1:]):
                # changing the last weight by multiplying its output_delta by it
                # hidden state matricies
                V -= np.outer(layers[layer_i]['s'],layer['output_delta'])*learning_rate/float(len(sent))

                #updating embeddings for each word by index
                U_i = sent[layer_i]
                U[U_i] -= layers[layer_i]['s']*alpha/float(len(sent))
                # updating W that is multiplied by the hidden state
                # in the main equation ^^ by multiplying the hidden layer weights
                # with the hidden delta

                W -= np.outer(layers[layer_i]['s'],layer['hidden_delta'])*learning_rate/ float(len(sent))

            if(iter % 1000 == 0):
                print("Perplexity:" + str(np.exp(loss/len(sent))))